bitsandbytes has issue that makes it need to be uninstalled and reinstalled each time
solution TBD

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # This will prompt you to enter the token

In [ ]:
# ============ 0. Imports and Setup ============
import json, os
from pathlib import Path
from typing import List, Dict, Any
import numpy as np
import torch
import torch.nn as nn
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)
from tabulate import tabulate
from sklearn.metrics import f1_score
from collections import Counter

In [ ]:
# ============ 2. Mount Drive and Load Data ============
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = Path("/content/drive/My Drive/SEMEVAL/data")
train_file = DATA_DIR / "training_set_task2.txt"
dev_file   = DATA_DIR / "dev_set_task2.txt"

def load_jsonl(path):
    return json.load(open(path, "r", encoding="utf-8"))

train_data = load_jsonl(train_file)
dev_data   = load_jsonl(dev_file)

In [ ]:
# ============ 3. Build Label Map ============
all_techniques = sorted({
    lbl["technique"].strip()
    for item in train_data + dev_data
    for lbl in item.get("labels", [])
})
label2id = {tech: i for i, tech in enumerate(all_techniques)}
id2label = {i: tech for tech, i in label2id.items()}

In [ ]:
# ============ 4. Tokenizer and Model ============
MODEL_NAME = "roberta-large"  # or "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    problem_type="multi_label_classification"
)

In [ ]:
# ============ 5. Preprocessing Functions ============
def tokenize_and_align_labels(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_offsets_mapping=True
    )
    labels = np.zeros((len(tokens["input_ids"]), len(label2id)), dtype=int)
    offsets = tokens.pop("offset_mapping")

    for span in example["labels"]:
        tech = span["technique"].strip()
        start_char, end_char = span["start"], span["end"]
        label_idx = label2id[tech]
        # label all tokens whose char span overlaps
        for i, (s,e) in enumerate(offsets):
            if s < end_char and e > start_char:
                labels[i, label_idx] = 1

    tokens["labels"] = labels.tolist()
    return tokens

# Build HuggingFace DatasetDict
raw_datasets = DatasetDict({
    "train": Dataset.from_list(train_data),
    "dev":   Dataset.from_list(dev_data)
})
tokenized = raw_datasets.map(tokenize_and_align_labels, batched=False)



label_counts = Counter()
total_tokens = 0

for ex in tokenized["train"]:
    for token_labels in ex["labels"]:
        for i, val in enumerate(token_labels):
            if val == 1:
                label_counts[i] += 1
        total_tokens += 1

# Convert to weights: inverse frequency
class_weights = []
for i in range(len(label2id)):
    count = label_counts.get(i, 1)
    weight = total_tokens / (len(label2id) * count)
    class_weights.append(weight)

# Convert to a tensor and move to the right device later
class_weights_tensor = torch.tensor(class_weights)

In [ ]:
# ============ 6. Official Partial‐Overlap Scorer ============
def compute_PRF(gold_spans, pred_spans, label):
    """
    P = avg_s in S ( sum_t overlap(s,t)/|s| ) / |S|
    R = avg_t in T ( sum_s overlap(s,t)/|t| ) / |T|
    F1 = 2PR/(P+R)
    """
    S = [s for s in pred_spans if s["technique"]==label]
    T = [t for t in gold_spans if t["technique"]==label]
    if not S or not T:
        return 0.0, 0.0, 0.0
    def C(s,t):
        if s["technique"]!=t["technique"]: return 0
        inter = max(0, min(s["end"],t["end"]) - max(s["start"],t["start"]))
        return inter / (s["end"]-s["start"])
    P = sum(max(C(s,t) for t in T) for s in S) / len(S)
    R = sum(max(C(s,t) for s in S) for t in T) / len(T)
    F1 = 2*P*R/(P+R) if (P+R)>0 else 0.0
    return P,R,F1

def evaluate_spans(examples, predictions, thresholds):
    # examples: list of dicts with "text","labels"
    # predictions: raw logits [batch, seq_len, num_labels]
    preds = torch.sigmoid(torch.tensor(predictions)).numpy()
    all_gold = []
    all_pred = []
    for ex,logits in zip(examples, preds):
        # reconstruct spans
        # get offsets
        enc = tokenizer(ex["text"], truncation=True, padding="max_length",
                        max_length=256, return_offsets_mapping=True)
        offsets = enc["offset_mapping"]
        gold = ex["labels"]
        pred = []
        for lbl_idx, thresh in thresholds.items():
            for i,(s,e) in enumerate(offsets):
                if s<e and logits[i,lbl_idx]>thresh:
                    pred.append({"technique": id2label[lbl_idx], "start":s,"end":e})
        all_gold.append(gold)
        all_pred.append(pred)
    # compute per-label and overall micro
    rows = []
    micro_tp=micro_fp=micro_fn=0
    for tech in all_techniques:
        P_list,R_list,F1_list=[],[],[]
        for gold,pred in zip(all_gold, all_pred):
            p,r,f = compute_PRF(gold,pred,tech)
            P_list.append(p); R_list.append(r); F1_list.append(f)
            # for micro: count as if each example a unit
            micro_tp += p
            micro_fp += (1-p)
            micro_fn += (1-r)
        rows.append([tech,
                     np.mean(P_list),
                     np.mean(R_list),
                     np.mean(F1_list)])
    microP = micro_tp/(micro_tp+micro_fp) if micro_tp+micro_fp>0 else 0
    microR = micro_tp/(micro_tp+micro_fn) if micro_tp+micro_fn>0 else 0
    microF = 2*microP*microR/(microP+microR) if microP+microR>0 else 0
    return rows, (microP,microR,microF)

In [ ]:
# ============ 7. Training Setup ============
class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        weights = class_weights_tensor.to(logits.device)
        loss_fct = nn.BCEWithLogitsLoss(pos_weight=weights)

        loss = loss_fct(
            logits.view(-1, logits.size(-1)),
            labels.view(-1, labels.size(-1)).float()
        )
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # optimize thresholds on dev batch
    thresholds = {}
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    labs  = np.array(labels)
    for i in range(len(all_techniques)):
        best_f, best_t = 0,0.5
        for t in np.arange(0.1,0.9,0.05):
            pr = (probs[:,:,i]>t).astype(int)
            f = f1_score(labs[:,:,i].flatten(), pr.flatten(), zero_division=0)
            if f>best_f: best_f, best_t = f,t
        thresholds[i]=best_t
    # now evaluate on this batch
    rows, micro = evaluate_spans(
        raw_datasets["dev"].select(range(len(probs))),
        logits, thresholds
    )
    print("\n=== Per-label results ===")
    print(tabulate(rows, headers=["Technique","P","R","F1"], floatfmt=".3f"))
    print(f"\n=== Micro Overall ===  P={micro[0]:.3f}  R={micro[1]:.3f}  F1={micro[2]:.3f}\n")
    return {"micro_f1": micro[2]}

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    num_train_epochs=10,
    logging_steps=50,
    save_strategy="no",
    report_to="none"
)

trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset= tokenized["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# ============ 8. Train and Evaluate ============
trainer.train()
trainer.evaluate()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/688 [00:00<?, ? examples/s]

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

<ipython-input-10-c04716738efe>:214: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultiLabelTrainer.__init__`. Use `processing_class` instead.
  trainer = MultiLabelTrainer(


Epoch,Training Loss,Validation Loss,Micro F1
1,0.125300,0.250413,0.051325
2,0.120600,0.211508,0.065530
3,0.078200,0.227135,0.063269
4,0.061300,0.297403,0.063340
5,0.051300,0.369734,0.061017
6,0.036000,0.434762,0.062711
7,0.021000,0.402700,0.060298
8,0.011000,0.466225,0.061900
9,0.009100,0.465834,0.057567
10,0.007700,0.487554,0.058121



=== Per-label results ===
Technique                                                P      R     F1
---------------------------------------------------  -----  -----  -----
Appeal to authority                                  0.026  0.032  0.028
Appeal to fear/prejudice                             0.006  0.016  0.008
Bandwagon                                            0.019  0.032  0.020
Black-and-white Fallacy/Dictatorship                 0.000  0.000  0.000
Causal Oversimplification                            0.011  0.016  0.013
Doubt                                                0.087  0.127  0.097
Exaggeration/Minimisation                            0.043  0.111  0.055
Flag-waving                                          0.047  0.063  0.052
Glittering generalities (Virtue)                     0.016  0.016  0.016
Loaded Language                                      0.229  0.366  0.265
Misrepresentation of Someone's Position (Straw Man)  0.022  0.032  0.026
Name calling/Labeling   


=== Per-label results ===
Technique                                                P      R     F1
---------------------------------------------------  -----  -----  -----
Appeal to authority                                  0.016  0.016  0.016
Appeal to fear/prejudice                             0.033  0.048  0.038
Bandwagon                                            0.000  0.000  0.000
Black-and-white Fallacy/Dictatorship                 0.000  0.000  0.000
Causal Oversimplification                            0.000  0.000  0.000
Doubt                                                0.039  0.048  0.042
Exaggeration/Minimisation                            0.033  0.048  0.034
Flag-waving                                          0.037  0.048  0.041
Glittering generalities (Virtue)                     0.000  0.000  0.000
Loaded Language                                      0.379  0.399  0.375
Misrepresentation of Someone's Position (Straw Man)  0.008  0.016  0.010
Name calling/Labeling   

{'eval_loss': 0.4875541627407074,
 'eval_micro_f1': 0.0581205355524153,
 'eval_runtime': 2.3135,
 'eval_samples_per_second': 27.232,
 'eval_steps_per_second': 6.916,
 'epoch': 10.0}